## Analyze whether TOBS varies more from year to year or from place to place.

In [1]:
import pandas as pd
import numpy as np
import urllib
import math

In [2]:
import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

In [3]:
import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

In [4]:
### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='SSSBSBBB'
meas='TOBS'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
print('keys from STAT=',STAT.keys())

#!ls -ld $data_dir/*.parquet

#read data
filename=data_dir+'/decon_%s_%s.parquet'%(file_index,meas)

df=sqlContext.read.parquet(filename)
print(df.count())
df.show(2)

('keys from STAT=', ['TMIN', 'TOBS', 'TMAX', 'SNOW', 'SNWD', 'PRCP'])
1199
+------------------+------------------+------------------+-------------------+---------+--------+--------+---------+-----------+------------------+------------------+------------------+------------------+-------------------+-----------+---------+------+--------------------+------+
|           coeff_1|           coeff_2|           coeff_3|            coeff_4|elevation|   label|latitude|longitude|measurement|             res_1|             res_2|             res_3|             res_4|           res_mean|    station|total_var|undefs|              vector|  year|
+------------------+------------------+------------------+-------------------+---------+--------+--------+---------+-----------+------------------+------------------+------------------+------------------+-------------------+-----------+---------+------+--------------------+------+
| 386.3015051306868|-443.7285977128111|181.22755885261162|-344.32458516518574| 

In [5]:
print df.columns

['coeff_1', 'coeff_2', 'coeff_3', 'coeff_4', 'elevation', 'label', 'latitude', 'longitude', 'measurement', 'res_1', 'res_2', 'res_3', 'res_4', 'res_mean', 'station', 'total_var', 'undefs', 'vector', 'year']


In [6]:
#extract longitude and latitude for each station
feature='coeff_1'
sqlContext.registerDataFrameAsTable(df,'weather')
#Features=', '.join(['coeff_1', 'coeff_2', 'coeff_3', 'elevation', 'latitude', 'longitude',\
#          'res_1', 'res_2', 'res_3', 'res_mean', 'year'])
Features='station, year, coeff_2'
Query="SELECT %s FROM weather"%Features
print(Query)
pdf = sqlContext.sql(Query).toPandas()
pdf.head()

SELECT station, year, coeff_2 FROM weather


station    year     coeff_2
0  USC00048331  1949.0 -443.728598
1  USC00262708  1982.0 -419.460529
2  USC00044713  1924.0  -91.839917
3  USC00265132  2002.0 -162.099123
4  USC00262708  1983.0 -194.648091

In [7]:
year_station_table=pdf.pivot(index='year', columns='station', values='coeff_2')
year_station_table.head(10)

station  USC00040931  USC00041018  USC00041462  USC00041912  USC00042334  \
year                                                                       
1901.0           NaN          NaN          NaN          NaN          NaN   
1902.0           NaN          NaN          NaN          NaN          NaN   
1903.0           NaN          NaN          NaN          NaN          NaN   
1904.0           NaN          NaN          NaN          NaN          NaN   
1905.0           NaN          NaN          NaN          NaN          NaN   
1906.0           NaN          NaN          NaN          NaN          NaN   
1907.0           NaN          NaN          NaN          NaN          NaN   
1908.0           NaN          NaN          NaN          NaN          NaN   
1909.0           NaN          NaN          NaN          NaN          NaN   
1910.0           NaN          NaN          NaN   322.071176          NaN   

station  USC00042338  USC00042467  USC00042500  USC00044288  USC00044713  \
year                                                                       
1901.0           NaN          NaN          NaN          NaN          NaN   
1902.0           NaN          NaN          NaN          NaN          NaN   
1903.0           NaN          NaN          NaN          NaN          NaN   
1904.0           NaN          NaN          NaN          NaN          NaN   
1905.0           NaN          NaN          NaN   614.131444          NaN   
1906.0           NaN          NaN          NaN   464.009970          NaN   
1907.0           NaN          NaN          NaN          NaN          NaN   
1908.0           NaN          NaN          NaN   613.536904          NaN   
1909.0           NaN          NaN          NaN          NaN          NaN   
1910.0           NaN          NaN          NaN          NaN          NaN   

station     ...       USS0015K03S  USS0017K04S  USS0019K04S  USS0019K07S  \
year        ...                                                            
1901.0      ...               NaN          NaN          NaN          NaN   
1902.0      ...               NaN          NaN          NaN          NaN   
1903.0      ...               NaN          NaN          NaN          NaN   
1904.0      ...               NaN          NaN          NaN          NaN   
1905.0      ...               NaN          NaN          NaN          NaN   
1906.0      ...               NaN          NaN          NaN          NaN   
1907.0      ...               NaN          NaN          NaN          NaN   
1908.0      ...               NaN          NaN          NaN          NaN   
1909.0      ...               NaN          NaN          NaN          NaN   
1910.0      ...               NaN          NaN          NaN          NaN   

station  USS0019K08S  USS0020K05S  USS0020K13S  USS0020K27S  USS0020K30S  \
year                                                                       
1901.0           NaN          NaN          NaN          NaN          NaN   
1902.0           NaN          NaN          NaN          NaN          NaN   
1903.0           NaN          NaN          NaN          NaN          NaN   
1904.0           NaN          NaN          NaN          NaN          NaN   
1905.0           NaN          NaN          NaN          NaN          NaN   
1906.0           NaN          NaN          NaN          NaN          NaN   
1907.0           NaN          NaN          NaN          NaN          NaN   
1908.0           NaN          NaN          NaN          NaN          NaN   
1909.0           NaN          NaN          NaN          NaN          NaN   
1910.0           NaN          NaN          NaN          NaN          NaN   

station  USS0020K31S  
year                  
1901.0           NaN  
1902.0           NaN  
1903.0           NaN  
1904.0           NaN  
1905.0           NaN  
1906.0           NaN  
1907.0           NaN  
1908.0           NaN  
1909.0           NaN  
1910.0           NaN  

[10 rows x 69 columns]

### Estimating the effect of the year vs the effect of the station

To estimate the effect of time vs. location on the first eigenvector coefficient we
compute:

* The average row: `mean-by-station`
* The average column: `mean-by-year`

We then compute the RMS before and after subtracting either  the row or the column vector.

In [8]:
def RMS(Mat):
    return np.sqrt(np.nanmean(Mat**2))

mean_by_year=np.nanmean(year_station_table,axis=1)
mean_by_station=np.nanmean(year_station_table,axis=0)
tbl_minus_year = (year_station_table.transpose()-mean_by_year).transpose()
tbl_minus_station = year_station_table-mean_by_station

print 'total RMS                   = ',RMS(year_station_table)
print 'RMS removing mean-by-station= ',RMS(tbl_minus_station)
print 'RMS removing mean-by-year   = ',RMS(tbl_minus_year)

total RMS                   =  249.16627865
RMS removing mean-by-station=  163.56249498
RMS removing mean-by-year   =  210.545871956


In [9]:
T=year_station_table
print 'initial RMS=',RMS(T)
for i in range(5):
    mean_by_year=np.nanmean(T,axis=1)
    T=(T.transpose()-mean_by_year).transpose()
    print i,'after removing mean by year    =',RMS(T)
    mean_by_station=np.nanmean(T,axis=0)
    T=T-mean_by_station
    print i,'after removing mean by stations=',RMS(T)

initial RMS= 249.16627865
0 after removing mean by year    = 210.545871956
0 after removing mean by stations= 122.828142199
1 after removing mean by year    = 117.168388624
1 after removing mean by stations= 116.283753777
2 after removing mean by year    = 116.041195087
2 after removing mean by stations= 115.952986148
3 after removing mean by year    = 115.914859469
3 after removing mean by stations= 115.896464298
4 after removing mean by year    = 115.886927202
4 after removing mean by stations= 115.881727567
